<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">


# Web Scraping for Indeed.com and Predicting Salaries

### Business Case Overview

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal wants you to

   - determine the industry factors that are most important in predicting the salary amounts for these data.

To limit the scope, your principal has suggested that you *focus on data-related job postings*, e.g. data scientist, data analyst, research scientist, business intelligence, and any others you might think of. You may also want to decrease the scope by *limiting your search to a single region.*

Hint: Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries.

**Goal:** Scrape your own data from a job aggregation tool like Indeed.com in order to collect the data to best answer this question.

---

### Directions

In this project you will be leveraging a variety of skills. The first will be to use the web-scraping and/or API techniques you've learned to collect data on data jobs from Indeed.com or another aggregator. Once you have collected and cleaned the data, you will use it to address the question above.

### Factors that impact salary

To predict salary the most appropriate approach would be a regression model.
Here instead we just want to estimate which factors (like location, job title, job level, industry sector) lead to high or low salary and work with a classification model. To do so, split the salary into two groups of high and low salary, for example by choosing the median salary as a threshold (in principle you could choose any single or multiple splitting points).

Use all the skills you have learned so far to build a predictive model.
Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to be able to extrapolate or predict the expected salaries for these listings.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10").

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters:

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from time import time, sleep
import numpy as np
import re
import sweetviz

In [2]:
pd.set_option('max_rows', 250, 'max_columns', 100)

In [3]:
url = "https://www.indeed.co.uk/jobs?q=data&l="

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

results = soup.find_all('div', class_='result')
results

[<div class="jobsearch-SerpJobCard unifiedRow row result" data-ci="358828147" data-empn="3916285824926666" data-jk="12e4f378a387252b" id="pj_12e4f378a387252b">
 <style>
 .jobcard_logo{margin:6px 0}.jobcard_logo img{width:auto;max-width:80px;max-height:30px}.jasxrefreshcombotst .jobcard_logo img{max-height:2rem;max-width:100%}
 </style>
 <h2 class="title">
 <a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0D-ce4GT5D_S5EFhkjqF6KrfkUuIXgHqfPbawmXBDzkY37HrqmV6D6RLNqm2tcsbdaQp2doy9pZTAEIorglQ0duyeI4PSuAl89XEv8VRo4y7rnC9T_cYGsuIRwcSHWkQ9cv7EnA7akGVQ1jNfOreirymEBdGzswpGF-pTBOx4MlRyMBu1ktGx18YxHnBF35FmHYjv6lksji_S_JoGPxYTzVNQjBmtmljVuS40ri8uq4EksIEhBUOVaLLT8BAgTMibejyPRHBAby_Qexe6D4vm-eFPtmpjGBzMVdDh_qEogIbcAYJa1zqy_l6kYlal67Axd16AJ7ky3bDjWD5hpGLmjq9ozB16HLDGrMO3eZfiNbv8iQwRErrRmX0HFXzu3kxhyIlrwKptw_Yz9bLSqKoERJ9f0lw8dpLrwnRCh-sfSdrLUDqzXWZpahIaT1YP4g13k=&amp;p=0&amp;fvj=1&amp;vjs=3" id="sja0" onclick="setRefineByCookie([]); sjoc('sja0', 1); c

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is in a `span` with `class='salaryText'`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element='jobTitle'`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 
- Decide which other components could be relevant, for example the region or the summary of the job advert.

### Write 4 functions to extract each item: location, company, job, and salary.

Example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it.
    - Remember to use `try/except` if you anticipate errors.
- **Test** the functions on the results above and simple examples.

In [4]:
title = []

def extract_title_from_result(results):
    for job in results:
        try:
            title.append(job.find(class_='jobtitle').text.replace('\n',''))
        except:
            title.append(np.nan)
extract_title_from_result(results)

In [5]:
print(title)

['Data Engineer', 'Data Entry Administrator', 'Data Analyst', 'SEN Data Quality Officer', 'Data Analyst', 'Data Entry Clerks', 'PUBLIC HEALTH DATA OFFICER', 'Data Scientist Intern', 'DATA ANALYST', 'Part Time role Data & Compliance Administrator', 'Admissions and Data Services Administrator', 'Social Research Data Collector', 'Junior Data Scientist', 'ITV Apprenticeship - Data Scientist Apprentice', 'Performance Data Analyst']


In [6]:
location = []

def extract_location_from_result(results):
    for job in results:
        try:
            location.append(job.find(class_="location").text)
        except:
            location.append(np.nan)
extract_location_from_result(results)

In [7]:
print(location)

['London W2 6LG', 'Stockport SK6 2SD', 'London', 'Lambeth', 'Home Based', 'Borough Station', 'Morden', 'London', 'London', 'Chichester', 'Cambridge', 'Northampton', 'London', 'London', 'Bexleyheath DA6']


In [8]:
company = []

def extract_company_from_result(results):
    for job in results:
        try:
            company.append(job.find(class_='company').text.replace('\n',''))
        except:
            company.append(np.nan)
extract_company_from_result(results)

In [9]:
print(company)

['HyperJar Limited', 'Simon Charles Auctioneers & Valuers', 'TikTok', 'Spencer Clarke Group', 'Aula Education', 'DirectTemping', 'Intec Select', 'Buzz Radar', 'Dr Martens', 'Capita Plc', 'University of Cambridge', 'Kantar', 'Novafutur', 'ITV Jobs', 'Global Project Resources Limited']


In [10]:
salary=[]

def extract_salary_from_result(results):
    for job in results:
        try:
            salary.append(job.find(class_='salary').text.replace('\n',''))
        except:
            salary.append(np.nan)
extract_salary_from_result(results)

In [11]:
print(salary)

['£35,000 - £50,000 a year', '£8.72 an hour', nan, '£18 - £23 an hour', '£300 - £400 a day', '£10.75 an hour', '£280 - £320 a week', nan, nan, '£19,000 a year', '£26,715 - £30,942 a year', '£80 - £100 a day', nan, '£21,032 a year', '£16 - £18 an hour']


In [12]:
date = []

def extract_date_from_result(results):
    for job in results:
        try:
            date.append(job.find(class_='date').text)
        except:
            date.append(np.nan)
extract_date_from_result(results)

In [13]:
print(date)

['Just posted', 'Just posted', '28 days ago', '5 days ago', '6 days ago', '14 days ago', '8 days ago', '27 days ago', '6 days ago', '10 days ago', '8 days ago', '11 days ago', '30+ days ago', 'Today', '3 days ago']


In [14]:
rating = []

def extract_rating_from_result(results):
    for job in results:
        try:
            rating.append(job.find(class_='ratingsContent').text.replace("\n",""))
        except:
            rating.append(np.nan)
extract_rating_from_result(results)   

In [15]:
print(rating)

[nan, nan, '4.0', nan, nan, nan, nan, nan, '4.1', '3.2', '4.2', '3.7', nan, '4.0', nan]


In [16]:
data_jobs = pd.DataFrame({'Title': title,
                        'Company': company,'Location': location,'Salary': salary,'Date': date,'Rating':rating
                         })
data_jobs

,Title,Company,Location,Salary,Date,Rating
0,Data Engineer,HyperJar Limited,London W2 6LG,"£35,000 - £50,000 a year",Just posted,NaN
1,Data Entry Administrator,Simon Charles Auctioneers & Valuers,Stockport SK6 2SD,£8.72 an hour,Just posted,NaN
2,Data Analyst,TikTok,London,NaN,28 days ago,4.0
3,SEN Data Quality Officer,Spencer Clarke Group,Lambeth,£18 - £23 an hour,5 days ago,NaN
4,Data Analyst,Aula Education,Home Based,£300 - £400 a day,6 days ago,NaN
5,Data Entry Clerks,DirectTemping,Borough Station,£10.75 an hour,14 days ago,NaN
6,PUBLIC HEALTH DATA OFFICER,Intec Select,Morden,£280 - £320 a week,8 days ago,NaN
7,Data Scientist Intern,Buzz Radar,London,NaN,27 days ago,NaN
8,DATA ANALYST,Dr Martens,London,NaN,6 days ago,4.1
9,Part Time role Data & Compliance Administrator,Capita Plc,Chichester,"£19,000 a year",10 days ago,3.2


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search.
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different.

In [ ]:
YOUR_CITY = ''

In [ ]:
url_template = "https://www.indeed.co.uk/jobs?q=data&l=United+Kingdom&start={}"
max_results_per_city = 5000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

results = []

for city in set(['London', 'Birmingham', 'Bristol', 'Leicester', 'Manchester',
    'Newcastle+Upon+Tyne', 'York', 'Sheffield', 'Nottingham', 'Liverpool', 'Portsmouth', 'Southampton',
    'Kingston', 'Edinburgh', 'Cambridge','Coventry','Plymouth','Wolverhampton','Glasgow','Leeds','Stoke-on-Trent',
    'Bradford','Brighton','Croydon']):
    for start in tqdm(range(0, max_results_per_city, 10)):
        r_ = requests.get(url.format(city, start))
        soup = BeautifulSoup(r_.text,'html.parser')
        results.append(soup)
        pass

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [ ]:
title = []
location = []
company = []
salary=[]
date = []
rating = []

for job in results:
        try:
            title.append(job.find(class_='jobtitle').text.replace('\n',''))
        except:
            title.append(np.nan)
        try:
            location.append(job.find(class_="location").text)
        except:
            location.append(np.nan)
        try:
            company.append(job.find(class_='company').text.replace('\n',''))
        except:
            company.append(np.nan)
        try:
            salary.append(job.find(class_='salary').text.replace('\n',''))
        except:
            salary.append(np.nan)
        try:
            date.append(job.find(class_='date').text)
        except:
            date.append(np.nan)
        try:
            rating.append(job.find(class_='ratingsContent').text.replace("\n",""))
        except:
            rating.append(np.nan)
            
job_scrape = pd.DataFrame({'Title': title,
                        'Company': company,'Location': location,'Salary': salary,'Date': date,'Rating':rating
                         })
job_scrape

In [ ]:
def data_overview(job_scrape, sort='Missing%'):
    list_of_numerics=job_scrape.select_dtypes(include=['float','int']).columns
    types= job_scrape.dtypes
    missing= round((job_scrape.isnull().sum()/job_scrape.shape[0]),3)*100
    overview = job_scrape.apply(lambda x: [round(x.min()), 
                                      round(x.quantile(0.5)),
                                      round(x.max()), 
                                      round(x.mean())] if x.name in list_of_numerics else x.nunique())
    outliers = job_scrape.apply(lambda x: sum(
                                     (x<(x.quantile(0.25)-1.5*(x.quantile(0.75)-x.quantile(0.25))))|
                                     (x>(x.quantile(0.75)+1.5*(x.quantile(0.75)-x.quantile(0.25))))
                                     if x.name in list_of_numerics else ''))
    explo = pd.DataFrame({'Types': types,
                          'Missing%': missing,
                          'Overview': overview,
                          'Outliers': outliers})
    return explo.sort_values(by=sort, ascending=False)

In [ ]:
data_overview(job_scrape)

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now.
1. Some of the entries may be duplicated.
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries.

In [ ]:
job_scrape

In [ ]:
job_scrape.drop(['Date', 'Rating'], axis=1, inplace=True)

In [ ]:
#removal of duplicates
df = job_scrape.drop_duplicates()

In [ ]:
#reset index
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

In [ ]:
df['Salary'] = df['Salary'].str.replace(r'a year$', '')

In [ ]:
df['Salary'] = df['Salary'].str.replace(r'an hour$', '')

In [ ]:
df['Salary'] = df['Salary'].str.replace(r'a day$', '')

In [ ]:
df['Salary'] = df['Salary'].str.replace(r'£', '')

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary.

In [ ]:
def sal_to_num(salary):
    sal= int
    '''Takes in a salary entry (as string) and extracts the digits as an integer.
    If the salary is in a range (eg 20,000-25,00) it returns the average'''
    
    try:
        sal = int(''.join(re.findall(r'([\d-])', salary)))
    except:
        sal = (int(''.join(re.findall(r'([\d-])', salary)).split("-")[0]) + int(''.join(re.findall(r'([\d-])', salary)).split("-")[1])) / 2
    
    
    return int(sal)
        
df.loc[:,'Salary'] = df.Salary.map(lambda x: sal_to_num(x)).copy()

In [ ]:
df

### Save your results as a CSV

In [ ]:
## YOUR CODE HERE

### Load in the the data of scraped salaries

In [ ]:
## YOUR CODE HERE

### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median).

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries.

In [ ]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [ ]:
## YOUR CODE HERE

### Create a classification model to predict High/Low salary. 


- Start by ONLY using the location as a feature.
- Use at least two different classifiers you find suitable.
- Remember that scaling your features might be necessary.
- Display the coefficients/feature importances and write a short summary of what they mean.
- Create a few new variables in your dataframe to represent interesting features of a job title (e.g. whether 'Senior' or 'Manager' is in the title).
- Incorporate other text features from the title or summary that you believe will predict the salary.
- Then build new classification models including also those features. Do they add any value?
- Tune your models by testing parameter ranges, regularization strengths, etc. Discuss how that affects your models.
- Discuss model coefficients or feature importances as applicable.

In [ ]:
## YOUR CODE HERE

### Model evaluation:

Your boss would rather tell a client incorrectly that they would get a lower salary job than tell a client incorrectly that they would get a high salary job. Adjust one of your models to ease his mind, and explain what it is doing and any tradeoffs.


- Use cross-validation to evaluate your models.
- Evaluate the accuracy, AUC, precision and recall of the models.
- Plot the ROC and precision-recall curves for at least one of your models.

In [ ]:
## YOUR CODE HERE

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### Bonus:

- Answer the salary discussion by using your model to explain the tradeoffs between detecting high vs low salary positions. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.
- Obtain the ROC/precision-recall curves for the different models you studied (at least the tuned model of each category) and compare.

In [ ]:
## YOUR CODE HERE

### Summarize your results in an executive summary written for a non-technical audience.
   
- Writeups should be at least 500-1000 words, defining any technical terms, explaining your approach, as well as any risks and limitations.

In [ ]:
## YOUR TEXT HERE IN MARKDOWN FORMAT 

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### BONUS

Convert your executive summary into a public blog post of at least 500 words, in which you document your approach in a tutorial for other aspiring data scientists. Link to this in your notebook.

In [ ]:
## YOUR LINK HERE IN MARKDOWN FORMAT 